# Drzewa sufiksów

1. Przyjmij następujący zbiór danych wejściowych:
    1. bbb\$
    2. aabbabd
    3. ababcd
    4. abaababaabaabaabab\$
    5. losowy tekst o długości 100 znaków,
    6. załączony plik.\
Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.
2. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.
3. Zaimplementuj algorytm konstruujący drzewo sufiksów.
4. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.
5. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
    1. Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (1p.),
    2. Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (1p.),
    3. Drzewo sufiksów - algorytm Ukkonena (3p).\
Dla załączonego tekstu czas wariantów 1 i 2 może być nieakceptowalnie długi - w tej sytuacji pomiń wyniki pomiarów dla tego tekstu.

6. Oczekiwany wynik ćwiczenia to kod źródłowy oraz raport w formie PDF.

In [1]:
from string import printable
from random import choice
from timeit import default_timer as timer
import pandas as pd

## ad 1. Przygotowanie danych wejściowych

In [2]:
dataA='bbb$' #$ - marker
dataB='aabbabd' #d - marker
dataC='ababcd' #d - marker
dataD='abaababaabaabaabab$' #$ - marker

In [3]:
dataE=''.join(choice(printable[1:]) for i in range(99))
dataE+=printable[0] #printable[0] ('0') - marker

In [4]:
with open("1997_714_head.txt") as file:
    dataF = file.read()
dataF += "$" #$ - marker

## ad 2. Suffix trie

### Bez suffix linków

In [5]:
class StaticTrieNode: 
    def __init__(self, parent):
        self.parent = parent
        self.children = dict()
    
    def suffixes(self):
        res=[]
        for child in self.children.keys():
            sts=self.children[child].__str__()
            for s in sts:
                res.append(child+s)
                
        if len(res)==0: return [""]
        return res
        
class StaticTrie:
    def __init__(self, text):
        start=timer()
        self.root = StaticTrieNode(None)
        for i in range(len(text)):
            suffix = text[i:]
            head, index = self.find(suffix)
            self.graft(head, suffix[index:])
        end=timer()
        self.time=end-start
    
    def __str__(self):
        return str(self.root.suffixes())

    def find(self, text):
        current_node = self.root
        idx = 0
        while idx < len(text) and text[idx] in current_node.children:
            current_node = current_node.children[text[idx]]
            idx += 1
        return current_node, idx

    def graft(self, node, text):
        for c in text:
            new_node = StaticTrieNode(node)
            node.children[c] = new_node
            node = new_node

    def pattern_search(self, pattern):
        if len(pattern) == 0:
            return True
        node, index = self.find(pattern)
        return index == len(pattern)

### Z suffix linkami

In [6]:
class DynamicTrieNode: 
    def __init__(self, parent, link=None):
        self.parent = parent
        self.children = dict()
        self.link=link
        
    def suffixes(self):
        res=[]
        for child in self.children.keys():
            sts=self.children[child].suffixes()
            for s in sts:
                res.append(child+s)
                
        if len(res)==0: return [""]
        return res
    
    def add_child(self, c):
        child = DynamicTrieNode(self)
        self.children[c] = child
        return child

    def add_link(self, node):
        self.link = node

        
class DynamicTrie: 
    def __init__(self, text):
        start=timer()
        self.root = DynamicTrieNode(None)
        
        deepest=self.root
        
        for i in range(len(text)):
            c=text[i]
            node=deepest
            deepest=None
            prev=None
            
            while(node and (c not in node.children)):
                child=node.add_child(c)
                node.children[c]=child
                #first iteration
                if (not deepest):
                    deepest=child
                #rest of iterations
                if (prev):
                    prev.add_link(child)
                if node == self.root:
                    child.add_link(self.root)
                prev=child
                node=node.link
            if(node):
                prev.link=node.children[c]
        end=timer()
        self.time=end-start
                
    def __str__(self):
        return str(self.root.suffixes())

    
    def pattern_search(self, pattern):
        node = self.root

        for c in pattern:
            if c not in node.children:
                return False

            node = node.children[c]

        return True

## ad 3. Suffix tree

In [7]:
class SuffixTreeNode: #to nie Ukkonen :((
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.children = dict()


class SuffixTree:
    def __init__(self, text):
        start=timer()
        self.root = SuffixTreeNode(0, len(text) - 1)
        self.text = text
        for i in range(len(text) - 1):
            suffix = text[i:]
            head, depth = self.find(suffix)
            self.graft(head, depth, i)
        end=timer()
        self.time=end-start

    def find(self, text, depth=0, node=None):
        if node is None:
            node = self.root
        next_node = node.children.get(text[0])
        if next_node is None:
            return node, depth

        next_node_text_len = next_node.end - next_node.start + 1
        for i in range(1, next_node_text_len):
            if self.text[next_node.start + i] != text[i]:
                stop_node = SuffixTreeNode(next_node.start, next_node.start + i - 1)
                next_node.start += i
                node.children[self.text[stop_node.start]] = stop_node
                stop_node.children[self.text[next_node.start]] = next_node
                return stop_node, depth + i
        return self.find(text[next_node_text_len:], next_node_text_len + depth, next_node)

    def graft(self, head, depth, i):
        new_node = SuffixTreeNode(depth + i, len(self.text) - 1)
        head.children[self.text[new_node.start]] = new_node

    def pattern_search(self, pattern, node=None):
        if len(pattern) == 0:
            return True
        if node is None:
            node = self.root
        next_node = node.children.get(pattern[0])
        if next_node is None:
            return False

        next_node_text_len = next_node.end - next_node.start + 1
        for i in range(1, next_node_text_len):
            if i == len(pattern):
                return True
            elif self.text[next_node.start + i] != pattern[i]:
                return False
        return self.pattern_search(pattern[next_node_text_len:], next_node)

## ad 4. Poprawność struktur

In [8]:
strieA=StaticTrie(dataA)
dtrieA=DynamicTrie(dataA)
treeA=SuffixTree(dataA)

strieB=StaticTrie(dataB)
dtrieB=DynamicTrie(dataB)
treeB=SuffixTree(dataB)

strieC=StaticTrie(dataC)
dtrieC=DynamicTrie(dataC)
treeC=SuffixTree(dataC)

strieD=StaticTrie(dataD)
dtrieD=DynamicTrie(dataD)
treeD=SuffixTree(dataD)

strieE=StaticTrie(dataE)
dtrieE=DynamicTrie(dataE)
treeE=SuffixTree(dataE)

In [9]:
strieF=StaticTrie(dataF)

In [10]:
dtrieF=DynamicTrie(dataF)

In [11]:
treeF=SuffixTree(dataF)

In [12]:
def test(strie, dtrie, tree, testok, testnotok):
    print("test StaticTrie:", end="\t")
    print("ok") if strie.pattern_search(testok) and not strie.pattern_search(testnotok) else print("not ok!!!")

    print("test DynamicTrie:", end="\t")
    print("ok") if dtrie.pattern_search(testok) and not dtrie.pattern_search(testnotok) else print("not ok!!!")

    print("test SuffixTree:", end="\t")
    print("ok") if tree.pattern_search(testok) and not tree.pattern_search(testnotok) else print("not ok!!!")

In [13]:
print("testing on data A")
test(strieA, dtrieA, treeA, "bb", "abk")

testing on data A
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


In [14]:
print("testing on data B")
test(strieB, dtrieB, treeB, "babd", "babdb")

testing on data B
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


In [15]:
print("testing on data C")
test(strieC, dtrieC, treeC, "abc", "abbbc")

testing on data C
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


In [16]:
print("testing on data D")
test(strieD, dtrieD, treeD, "baababa", "baababaaaaa")

testing on data D
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


In [17]:
print("testing on data E")
test(strieE, dtrieE, treeE, dataE[3:50], "00")

testing on data E
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


In [18]:
print("testing on data F")
test(strieF, dtrieF, treeF, "163, Nr 90, poz. 419, Nr 113, poz.", "poz. 419, Nr 113ala ma kota")

testing on data F
test StaticTrie:	ok
test DynamicTrie:	ok
test SuffixTree:	ok


## ad 5. Porównanie czasów

In [19]:
times={'A': {"trie (no links)":dtrieA.time, "trie (links)":dtrieA.time, "suffix tree": treeA.time},
                'B': {"trie (no links)":dtrieB.time, "trie (links)":dtrieB.time, "suffix tree": treeB.time},
                'C': {"trie (no links)":dtrieC.time, "trie (links)":dtrieC.time, "suffix tree": treeC.time},
                'D': {"trie (no links)":dtrieD.time, "trie (links)":dtrieD.time, "suffix tree": treeD.time},
                'E': {"trie (no links)":dtrieE.time, "trie (links)":dtrieE.time, "suffix tree": treeE.time},
                'F': {"trie (no links)":dtrieF.time, "trie (links)":dtrieF.time, "suffix tree": treeF.time}}

In [20]:
df_times=pd.DataFrame(times)
display(df_times)

,A,B,C,D,E,F
trie (no links),0.000048,0.000021,0.000015,0.000086,0.006526,14.445446
trie (links),0.000048,0.000021,0.000015,0.000086,0.006526,14.445446
suffix tree,0.000013,0.000014,0.000010,0.000140,0.000188,0.042381
